### Installations, if needed

In [ ]:
# Install the right version of spaCy
!pip install spacy==2.0.12 # Above 2.0.12 doesn't seem work with the neuralcoref resolution (at least 2.0.13 and 2.0.16 don't)

In [ ]:
# Install the large Neural Coref model
!pip install https://github.com/huggingface/neuralcoref-models/releases/download/en_coref_lg-3.0.0/en_coref_lg-3.0.0.tar.gz # This is the coref language model

### Importing Libraries

In [7]:
import spacy
import en_coref_lg
from spacy import displacy
import pandas as pd
import numpy as np
import networkx as nx
import time
from graphviz import Source
import matplotlib.pyplot as plt
import os

### Loading and previewing our export from OntoNotes5

In [8]:
JSON_FILENAME = 'ner_output_1.json'
FILEPATH_TO_JSON = "onto_sql_output/"

onto_import = pd.read_json(FILEPATH_TO_JSON + JSON_FILENAME)

In [9]:
onto_import.head()

,document_text,ner_end_word_index,ner_start_word_index,ner_string,ner_type,onto_sentence_parse,sent@doc_id,sentence_string
0,The explosion in Yemen did not help an already...,32,32,Israeli,NORP,(TOP (S (S (NP-SBJ (NP (DT The) (NN explosion)...,0@bn/abc/00/abc_0008@0008@abc@bn@en@on,The explosion in Yemen did not help an already...
1,The explosion in Yemen did not help an already...,34,34,Palestinian,NORP,(TOP (S (S (NP-SBJ (NP (DT The) (NN explosion)...,0@bn/abc/00/abc_0008@0008@abc@bn@en@on,The explosion in Yemen did not help an already...
2,The explosion in Yemen did not help an already...,3,3,Yemen,GPE,(TOP (S (S (NP-SBJ (NP (DT The) (NN explosion)...,0@bn/abc/00/abc_0008@0008@abc@bn@en@on,The explosion in Yemen did not help an already...
3,The explosion in Yemen did not help an already...,37,37,Egypt,GPE,(TOP (S (S (NP-SBJ (NP (DT The) (NN explosion)...,0@bn/abc/00/abc_0008@0008@abc@bn@en@on,The explosion in Yemen did not help an already...
4,The explosion in Yemen did not help an already...,42,39,the next several days,DATE,(TOP (S (S (NP-SBJ (NP (DT The) (NN explosion)...,0@bn/abc/00/abc_0008@0008@abc@bn@en@on,The explosion in Yemen did not help an already...


In [10]:
onto_import.loc[0].sentence_string

"The explosion in Yemen did not help an already tense situation in the Middle East , but there is the very slightest hope tonight , because there 's word of a possible Israeli / Palestinian summit in Egypt in the next several days ."

### Loading the Large Neural Coref spaCy Model

In [11]:
nlp = en_coref_lg.load()

#### Using it to loop through dependency parsing for a selection of sentences

In [12]:
# Establish which sentence rows we want to work through right now
SENT_MIN = 0
SENT_MAX = 500

# Check the time and start parsing via spaCy
start_time = time.time()
onto_import["spacy_parse"] = onto_import.loc[SENT_MIN:SENT_MAX,:].apply(lambda x: nlp(x["sentence_string"]), axis=1)

# Calculate the duration 
duration = time.time() - start_time
print("Applying the spaCy pipeline took {0:.2f} seconds".format(duration))

Applying the spaCy pipeline took 10.32 seconds


### Viewing some example text, with highlighted named entities

In [ ]:
# Choose an entry integer to see its text and the parse below.
ENTRY = 490

displacy.render(onto_import.loc[ENTRY,"spacy_parse"], jupyter=True, style='ent')

#### Viewing dependencies

In [ ]:
displacy.render(onto_import.loc[ENTRY,"spacy_parse"], jupyter=True, style='dep')

#### Identifying some useful attributes of the spaCy tokens

In [ ]:
# This gives us the dependency type

onto_import.loc[490,"spacy_parse"][13].head

In [ ]:
# This gives us the token of its head, from which we can call other attributes of the head.

onto_import.loc[490,"spacy_parse"][13].head.i

In [ ]:
# If we just want the string of the head, that's here:

onto_import.loc[0,"spacy_parse"][32].head.text

In [ ]:
# If we want the index of the head within the sentence (to find when a multi-word NE
# depends on something outside of that NE phrase)

onto_import.loc[0,"spacy_parse"][32].head.i

### Creating a Dictionary with a Graph for Each NER Type

#### Establish dictionary to hold graphs and define a function to graph a row 

In [72]:
graphs_dict = dict()

def graph_row(df_row):
    
    NO_OF_GENERATIONS = 2
    
    root_node = str(df_row["ner_type"])

    # Retrieve our Directed Graph for this NE Type or create a new one
    G = graphs_dict.get(root_node, nx.DiGraph())
    
    # For each row, add a node for the Named Entity's type
    G.add_node(root_node)
    node_weight = G.nodes[root_node].get('weight', 0)
    G.node[root_node]['weight'] = node_weight + 1
    G.node[root_node]['xlabel'] = G.node[root_node]['weight']

    # Let's assume the head is the last word of the phrase
    # and get that Token from the spaCy parse:
    
    head_index = df_row["ner_end_word_index"]
    head_of_phrase = df_row["spacy_parse"][head_index]
    
    nodes_needing_head_branches = [(head_of_phrase, root_node)]
    next_head_nodes = []
    nodes_needing_child_branches = [(head_of_phrase, root_node)]
    next_child_nodes = []
    
    current_gen = 1
    
    while current_gen <= NO_OF_GENERATIONS:
        
        for (node, child_label) in nodes_needing_head_branches:
            try: 
                # Get the explanation of its dependency type in this usage
                relation = spacy.explain(node.dep_)

                # If no explanation, revert to the raw dependency type.
                if relation is None:
                    relation = node.dep_

                # Trying to catch and diagnose some problem cases
                elif relation == "punctuation":
                    print("NE '{1}' marked as punctuation in sentence '{0}'".format(df_row["sentence_string"], df_row["ner_string"]))
                    print(" --- ")
                elif relation == "determiner":
                    print("NE '{1}' marked as determiner in sentence '{0}'".format(df_row["sentence_string"], df_row["ner_string"]))
                    print(" --- ")

                # Object of preposition doesn't do much, so let's see what's on the other side of that.
                elif relation == "object of preposition":
                    relation = "head of prep phrase"
                    # move to the preposition so we get its head later on when adding node
                    node = node.head
                
                # differentiating head-focused edges from child-focused edges
                intermediary_node_label = "head g{0} {1}".format(current_gen, relation)

                # Add a node for the relation, and connect that to the main entity.
                # Add the weights to the nodes and edges.
                G.add_node(intermediary_node_label)
                relation_weight = G.node[intermediary_node_label].get('weight', 0)
                G.node[intermediary_node_label]['weight'] = relation_weight + 1
                
                # Add a node from the dependency type to the entity's head and add weights
                norm = node.head.norm_
                G.add_node(norm)
                norm_edge_weight = G.node[norm].get('weight', 0)
                G.node[norm]['weight'] = norm_edge_weight + 1
                
                G.add_edge(child_label, intermediary_node_label, label="head")
                
                G.add_edge(intermediary_node_label, norm)
                
                # Add the next round for the next generation
                if node.head != node:
                    next_head_nodes.append((node.head, norm))
                
            except:
                print("passed in head")
                pass
            
            # Move the next round into the queue and clear it
            nodes_needing_head_branches = next_head_nodes
            next_head_nodes = []

            for (node, parent_label) in nodes_needing_child_branches:
                for child in node.children:

                    # Get the relation of the child node to this one.
                    relation = spacy.explain(child.dep_)

                    # If no explanation, revert to the raw dependency type.
                    if relation is None:
                        relation = child.dep_

                    if relation == 'punctuation':
                        break

                    # Differentiate these relations from head relations
                    # and add the node and its weights
                    intermediary_node_label = "child g{0} {1}".format(current_gen, relation)
                    G.add_node(intermediary_node_label)
                    relation_weight = G.nodes[intermediary_node_label].get('weight', 0)
                    G.node[intermediary_node_label]['weight'] = relation_weight + 1
                    
                    # Add the child as normed, and add its edge and weights
                    child_norm = child.norm_
                    G.add_node(child_norm)
                    leaf_weight = G.node[child_norm].get('weight', 0)
                    G.node[child_norm]['weight'] = leaf_weight + 1

                    # add edge between the parent node and this relation and weights
                    G.add_edge(parent_label, intermediary_node_label, label="child")
                    
                    G.add_edge(intermediary_node_label, child_norm)

                    # Queue up the children for the next generation
                    for childs_child in child.children:
                        next_child_nodes.append((childs_child, child_norm))      
                
                # Move the children into the queue and clear it.
                nodes_needing_child_branches = next_child_nodes
                next_child_nodes = []
        
        # Increment the generation
        current_gen += 1

    graphs_dict[root_node] = G

#### Applying that to our selected rows of our dataframe

In [73]:
%time
_ = onto_import.loc[SENT_MIN:SENT_MAX,:].apply(lambda x: graph_row(x), axis=1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs
Adding summit, head of headleaf summit
Adding word, head of headleaf word
Adding summit, head of headleaf summit
Adding word, head of headleaf word
Adding explosion, head of headleaf explosion
Adding help, head of headleaf help
Adding summit, head of headleaf summit
Adding word, head of headleaf word
Adding summit, head of headleaf summit
Adding word, head of headleaf word
Adding is, head of headleaf is
Adding help, head of headleaf help
Adding situation, head of headleaf situation
Adding help, head of headleaf help
Adding signed, head of headleaf signed
Adding signed, head of headleaf signed
Adding granting, head of headleaf granting
Adding bill, head of headleaf bill
Adding signed, head of headleaf signed
Adding status, head of headleaf status
Adding review, head of headleaf review
Adding imports, head of headleaf imports
Adding tariffs, head of headleaf tariffs
Adding agreed, head of headleaf agreed
Adding goods, head of

#### Calculate the Log Probability of the Named Entity Types

In [81]:
def normalize_as_logp(successor_nodes, G):
    log_total = np.log(sum([G.node[node]['weight'] for node in successor_nodes]))
    for n in successor_nodes:
        G.node[n]['lp'] = np.log(G.node[n]['weight']) - log_total

def calc_lps(G):
    for node in nx.nodes(G):
        if G.successors(node):
            [normalize_as_logp(succ[1], G) for succ in nx.bfs_successors(G, node)]

In [82]:
%time

log_graphs_dict = {key:calc_lps(value) for (key,value) in graphs_dict.items()}

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


/anaconda/envs/ear/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


#### Draw the graphs for the Named Entity Types

In [83]:
for key, value in graphs_dict.items():
    
    graph_filepath = 'NER_Type_Graphs/'
    graph_filename = 'G_{0}'.format(key)

    # Write our graph to DOT format to be read and visualized by GraphViz
    nx.drawing.nx_pydot.write_dot(value, "{0}.dot".format(graph_filepath + graph_filename))
    
    # Write the graphs via neato
    os.system("neato -Tsvg -Goverlap=false -Gsplines=true -Gsep=0.1 {0}.dot -o {0}.svg".format(graph_filepath + graph_filename))

### Graphing a Candidate Named Entity

In [ ]:
# Defining helper functions to build the candidate graphs

def reconcile_ents_and_clusters(doc):
    """"Reconcile the coreference and entities lists into a
        a single dict of graphs to make.
        
        Keys are (start.idx, end.idx) tuples.
        Values are (spaCy.Span, graph_id) tuples."""
    
    # Keys are (start.idx, end.idx) tuples.
    # Values are (spaCy.Span, graph_id) tuples."
    occurence_ind  = {}
    
    cluster_offset = 0
    if doc._.has_coref:
        cluster_offset = len(doc._.coref_clusters)
        for cluster_idx, cluster in enumerate(doc._.coref_clusters):
            for mention in cluster:
                key = (mention.start, mention.end)
                occurence_ind[key] = (mention, cluster_idx)
    
    # Now let's see if each ent is in there. If not, we'll add it to
    # our cluster list.
    new_cluster_idx = 0
    
    for ent_ind, ent in enumerate(doc.ents):
        key = (ent.start, ent.end)
        try:
            occurence_ind[key]
        except:
            occurence_ind[key] = (ent, cluster_offset + new_cluster_idx)
            new_cluster_idx += 1
    return occurence_ind

def graph_entity(ent, doc, G, root_node):
    
    NO_OF_GENERATIONS = 2
    
    # Assume the head of the phrase, if it is a phrase, is the last word
    # in the phrase.
    head_of_phrase = ent[-1]
    
    nodes_needing_head_branches = [(head_of_phrase, root_node)]
    next_head_nodes = []
    nodes_needing_child_branches = [(head_of_phrase, root_node)]
    next_child_nodes = []
    
    while NO_OF_GENERATIONS >= 0:
        NO_OF_GENERATIONS -= 1
        
        for (node, child_label) in nodes_needing_head_branches:
            try: 
                # Get the explanation of its relation arc in this usage
                relation = spacy.explain(node.dep_)
                # If no explanation, revert to the raw dependency type.
                if relation is None:
                    relation = node.dep_

                # Object of preposition doesn't do much, so let's see what's on the other side of that.
                elif relation == "object of preposition":
                    relation = "head of prep phrase"
                    # move to the preposition so we get its head later on when adding node
                    node = node.head
                    
                intermediary_node_label = "head " + relation

                # Add a node for the relation, and connect that to the main entity
                G.add_node(intermediary_node_label)
                G.add_edge(child_label, intermediary_node_label, label="head")

                # Add a node from the relation to the entity's head, and connect that
                # to the relation type
                normed_head = node.head.norm_
                G.add_node(normed_head)
                G.add_edge(intermediary_node_label, normed_head)
                
                if node.head != node:
                    next_head_nodes.append((node.head, node.head.text))
            except:
                print("passed in head")
                pass
            
        nodes_needing_head_branches = next_head_nodes
        next_head_nodes = []
        
        for (node, parent_label) in nodes_needing_child_branches:
            try:
                for child in node.children:
                    relation = spacy.explain(child.dep_)
                    # If no explanation, revert to the raw dependency type.
                    if relation is None:
                        relation = node.dep_
                    intermediary_node_label = "child " + relation

                    G.add_node(intermediary_node_label)
                    G.add_edge(parent_label, intermediary_node_label, label="child")
                    print("added child edge from {0} to {1}".format(parent_label, intermediary_node_label))
                    
                    normed_child = child.norm_
                    G.add_node(normed_child)
                    print("adding child node: {0}".format(normed_child))
                    G.add_edge(intermediary_node_label, normed_child)
                    
                    for childs_child in child.children:
                        next_child_nodes.append((childs_child, node.childs_child.text))
            except:
                print("passed in child")
                pass                
        nodes_needing_child_branches = next_child_nodes
        next_child_nodes = []
    return G
    
def graph_candidates_in_doc(candidate_text):
    
    doc = nlp(candidate_text)
    
    clustered_ents = reconcile_ents_and_clusters(doc)
    
    # Initialize a graph for each clustered_ent
    candidate_graphs = dict()
    
    for ((start_idx, end_idx), (ent,graph_idx)) in clustered_ents.items():
        
        # Get the cluster's existing graph from previous mentions
        # or create a new one.
        G = candidate_graphs.get(graph_idx, nx.DiGraph())
        
        # Make sure we have our root. No harm done if it already exists.
        # If it's a cluster, we get the Span of the most representative
        # mention in the cluster
        try:
            root_node = doc._.coref_clusters[graph_idx].main.text
        # If it's not, we just use the ent name
        except:
            root_node = ent.text
        G.add_node(root_node)
        
        # A helper function adds the rest of the graph
        print("\nGraphing entity: {0}".format(ent.text))
        candidate_graphs[graph_idx] = graph_entity(ent, doc, G, root_node)
    
    #graph_entity(ent, doc) for ent in doc.ents]
    return candidate_graphs

#### Applying the functions on a selected article

In [ ]:
TEST_ENTRY = 90
test_candidate_text = onto_import.loc[TEST_ENTRY,"document_text"]
print(test_candidate_text)

test_graphs = graph_candidates_in_doc(test_candidate_text)

#### Draw the candidate graphs for that article

In [ ]:
for key, value in test_graphs.items():
    
    graph_filepath = 'NER_Type_Graphs/'
    graph_filename = 'G_' + str(key)

    # Write our graph to DOT format to be read and visualized by GraphViz
    nx.drawing.nx_pydot.write_dot(value, "{0}.dot".format(graph_filepath + graph_filename))
    
    os.system("neato -Tsvg -Goverlap=false -Gsplines=true -Gsep=0.1 {0}.dot -o {0}.svg".format(graph_filepath + graph_filename))

#### Holding area for test to show that the candidate doc ents are all in the reconciled list

In [ ]:
# Some tests
TEST_ENTRY = 90
doc = nlp(onto_import.loc[TEST_ENTRY,"document_text"])

# Testing to make sure all the ents are present in the reconciled list
reconciled = reconcile_ents_and_clusters(doc)
for key in [(ent.start, ent.end) for ent in doc.ents]:
    assert key in reconciled.keys()